In [4]:
import wetter_api as wt
# import waldbrand
import model_eval as me
from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# from sklearn.svm import SVR
# from sklearn.metrics import mean_squared_error, r2_score

In [8]:
parameter_list = [
    wt.DwdObservationParameter.MONTHLY.PRECIPITATION_HEIGHT,
    wt.DwdObservationParameter.MONTHLY.CLIMATE_SUMMARY.WIND_FORCE_BEAUFORT,
    wt.DwdObservationParameter.MONTHLY.SUNSHINE_DURATION,
]
# Dictionary with short names as keys and full names as values
shortname_to_state = {
    "BW": "Baden-Württemberg",
    "BY": "Bayern",
    "BE": "Berlin",
    "BB": "Brandenburg",
    "HB": "Bremen",
    "HH": "Hamburg",
    "HE": "Hessen",
    "MV": "Mecklenburg-Vorpommern",
    "NI": "Niedersachsen",
    "NW": "Nordrhein-Westfalen",
    "RP": "Rheinland-Pfalz",
    "SL": "Saarland",
    "SN": "Sachsen",
    "ST": "Sachsen-Anhalt",
    "SH": "Schleswig-Holstein",
    "TH": "Thüringen",
}

# Example usage
shortname = "BB"
state = shortname_to_state.get(shortname)

In [6]:
# shortname = "BW"
# state = shortname_to_state.get(shortname)
# merged_data = me.prepare_data(parameter_list, state)
import pandas as pd
import common_paths
merged_data = pd.read_csv(common_paths.DATA.joinpath("dwd/Brandenburg.csv"))

In [9]:
test_size=0.2
random_state=42
str_parameter_list = [str(param) for param in parameter_list]
feature_list = ["Year", "Month"] + str_parameter_list
X = merged_data[feature_list]
y = merged_data["nFires"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)
# results = evaluate_model(X_train, X_test, y_train, y_test)

In [10]:
# How to define this?
param_grid = {
    "max_depth": list(range(5, 21, 5)),  # Smaller range for max depth
    "max_features": [1, 2, 3],  # Only up to the number of features you have
}
rf = RandomForestRegressor()

grid_search = GridSearchCV(
    estimator=rf, param_grid=param_grid, cv=3, scoring="neg_mean_squared_error"
)


grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

In [55]:
from sklearn.metrics import mean_squared_error, r2_score
# Evaluate the best model
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"R2 Score: {r2}")

MSE: 0.0008343595342124447
R2 Score: 0.9999995231006401


In [33]:
me.evaluate_model(X_train, X_test, y_train, y_test, "Gradient Boosting Regression")

{'model': GradientBoostingRegressor(),
 'mse': 151.93781015930125,
 'r2_score': 0.9131560898604607}

In [32]:
me.evaluate_model(X_train, X_test, y_train, y_test, "Linear Regression")

{'model': LinearRegression(),
 'mse': 720.8955480615789,
 'r2_score': 0.5879538600022325}

In [31]:
me.evaluate_model(X_train, X_test, y_train, y_test, "Ridge Regression")

{'model': Ridge(), 'mse': 720.8955480560656, 'r2_score': 0.5879538600053837}

In [34]:
me.evaluate_model(X_train, X_test, y_train, y_test, "Decision Tree Regression")

{'model': DecisionTreeRegressor(), 'mse': 0.0, 'r2_score': 1.0}